In [3]:
import dlib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

In [4]:
predictor_path = "./shape_predictor_68_face_landmarks.dat"
face_rec_model_path = "./dlib_face_recognition_resnet_model_v1.dat"
 
detector = dlib.get_frontal_face_detector() #a detector to find the faces
sp = dlib.shape_predictor(predictor_path ) #shape predictor to find face landmarks
facerec = dlib.face_recognition_model_v1(face_rec_model_path) #face recognition model

In [5]:
img_data_train=np.zeros((5000,128))
for i in range (5000):
    img = dlib.load_rgb_image('../datasets/celeba/img/' + str(i) + '.jpg')
    dets = detector(img, 1)  #Extract the face area in the picture
    for k, d in enumerate(dets):
        shape = sp(img, d)
        face_descriptor = facerec.compute_face_descriptor(img, shape)
        img_data_train[i,:]=face_descriptor

In [7]:
img_data_train_df = pd.DataFrame(img_data_train)
img_data_train_df.to_csv('face_feature_train.csv')

In [6]:
print(np.shape(img_data_train))

(5000, 128)


In [8]:
label_train=pd.read_table('../datasets/celeba/labels.csv')

In [9]:
img_data_test=np.zeros((1000,128))
for i in range (1000):
    img = dlib.load_rgb_image('../datasets/celeba_test/img/' + str(i) + '.jpg')
    dets = detector(img, 1)  #Extract the face area in the picture
    for k, d in enumerate(dets):
        shape = sp(img, d)
        face_descriptor = facerec.compute_face_descriptor(img, shape)
        img_data_test[i,:]=face_descriptor

In [10]:
img_data_test_df = pd.DataFrame(img_data_test)
img_data_test_df.to_csv('face_feature_test.csv')
print(np.shape(img_data_test))

(1000, 128)


In [11]:
label_test=pd.read_table('../datasets/celeba_test/labels.csv')

In [12]:
x_train = img_data_train
y_train= label_train['gender']
x_test = img_data_test
y_test= label_test['gender']

In [13]:
x_train = StandardScaler().fit_transform(x_train)
x_test = StandardScaler().fit_transform(x_test)

In [14]:
clf=SVC(gamma='auto')
clf.fit(x_train,y_train)
y_pred =  clf.predict(x_test)

In [15]:
print('Accuracy Score on train data: ', accuracy_score(y_true=y_train, y_pred=clf.predict(x_train)))
print('Accuracy Score on test data: ', accuracy_score(y_true=y_test, y_pred=y_pred))

Accuracy Score on train data:  0.978
Accuracy Score on test data:  0.969


In [17]:
import pickle

#save model
pickle.dump(clf,open("SVM_gender_dlib.dat","wb"))  

# load model
loaded_model = pickle.load(open("SVM_gender_dlib.dat","rb"))

y_pred=loaded_model.predict(x_test)